# Taller 3. Filtrado de marcadores moleculares tipo SNP
Genética de rasgos complejos 2024-01 - Docente Johana Carolina Soto Sedano

__Hans D. Escobar H.__

Las instrucciones están dadas para hacerlo en Excel. Lo haré con Python para practicar, también agregaré los pasos extra que considere pertinentes. 

In [17]:
from pathlib import Path
import pandas as pd

In [18]:
data_path = Path("../Private_Data/GBS.txt")
raw_df = pd.read_csv(str(data_path), sep="\t", index_col=0, dtype=str)

## 1. Explorando los datos

- Revisando los campos y estructura de la tabla:

In [19]:
width = 10
for line in range(0, len(raw_df.columns), width):
    print("{}".format(list(raw_df.columns[line : line + width])))

['alleles', 'chrom', 'pos', 'strand', 'assembly', 'center', 'protLSID', 'assayLSID', 'panelLSID', 'QCcode']
['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10']
['G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20']
['G21', 'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30']
['G31', 'G32', 'G33', 'G34', 'G35', 'G36', 'G37', 'G38', 'G39', 'G40']
['G41', 'G42', 'G43', 'G44', 'G45', 'G46', 'G47', 'G48', 'G49', 'G50']
['G51', 'G52', 'G53', 'G54', 'G55', 'G56', 'G57', 'G58', 'G59', 'G60']
['G61', 'G62', 'G63', 'G64', 'G65', 'G66', 'G67', 'G68', 'G69', 'G70']
['G71', 'G72', 'G73', 'G74', 'G75', 'G76', 'G77', 'G78', 'G79', 'G80']
['G81', 'G82', 'G83', 'G84', 'G85', 'G86', 'G87', 'G88', 'G89', 'G90']
['G91', 'G92', 'G93', 'G94', 'G95', 'G96', 'G97', 'G98', 'G99', 'G100']
['G101', 'G102', 'G103', 'G104', 'G105', 'G106', 'G107', 'G108', 'G109', 'G110']
['G111', 'G112', 'G113', 'G114', 'G115', 'G116', 'G117', 'G118', 'G119', 'G120']
['G121', 'G122', 'G123', 'G1

In [20]:
print("Total de filas: {}\n".format(len(raw_df)))
raw_df.head(n=3)

Total de filas: 83830



,alleles,chrom,pos,strand,assembly,center,protLSID,assayLSID,panelLSID,QCcode,...,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
rs,,,,,,,,,,,,,,,,,,,,,
SNP1,C/T,ST4.03ch01,2597267,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP2,A/G,ST4.03ch01,2631717,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP3,C/A,ST4.03ch01,2631718,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC


- ¿Hay loci multialélicos?

In [21]:
print("Tipos de SNP presentes:\n\n{}".format(raw_df["alleles"].unique()))

Tipos de SNP presentes:

['C/T' 'A/G' 'C/A' 'G/A' 'T/C' 'A/C' 'G/T' 'G/C' 'A/T' 'T/G' 'T/A' 'C/G']


- ¿De que forma son los genotipos presentes?

In [22]:
def get_unique_genotypes(df: pd.DataFrame):
    genotypes = set()
    for colname in df.columns[10:]:
        genotypes.update(df[colname].unique())
    return genotypes


unique_genotypes = get_unique_genotypes(raw_df)
print("{} genotipos únicos:\n\n{}".format(len(unique_genotypes), unique_genotypes))

17 genotipos únicos:

{'AA', 'GT', 'AG', 'NN', 'TG', 'CA', 'CG', 'TC', 'CT', 'GA', 'TT', 'GC', 'AC', 'TA', 'GG', 'AT', 'CC'}


Están los 16 esperados $card(\{A, G, T, C\} \times \{A, G, T, C\})$, mas uno representando genotipo desconocido $(NN)$. No hay datos perdidos (NA).

## 2. Filtrando los datos

In [23]:
class FilterDescription:
    kept_markers: int
    message: str

    def __init__(self, message: str, kept_markers: int):
        self.kept_markers = kept_markers
        self.message = message

    def __str__(self):
        return f"{self.message}: {self.kept_markers}"


filter_summary: list[FilterDescription] = list()

- Con base a la proporción de no genotipados

In [24]:
# Preserva los que tienen igual o mayor proporción que threshold
def missing_proportion_filter(df, threshold=0.85):
    def filter_NN_and_NaN(row, threshold):
        counts = row.iloc[10:].value_counts(normalize=True, dropna=False)
        return counts.get("NN", 0) + counts.get(pd.NA, 0) < threshold

    return df[df.apply(filter_NN_and_NaN, axis=1, threshold=threshold)]


missing_threshold = 0.85
preprocessed_df = missing_proportion_filter(raw_df, missing_threshold)
filter_summary.append(
    FilterDescription(
        f"Marcadores con frecuencia de datos faltantes (NN o sin datos) < {1-missing_threshold:.3f}",
        len(preprocessed_df),
    )
)
preprocessed_df

,alleles,chrom,pos,strand,assembly,center,protLSID,assayLSID,panelLSID,QCcode,...,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
rs,,,,,,,,,,,,,,,,,,,,,
SNP1,C/T,ST4.03ch01,2597267,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP2,A/G,ST4.03ch01,2631717,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP3,C/A,ST4.03ch01,2631718,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP4,G/A,ST4.03ch01,2631725,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,2631732,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNP83826,A/G,ST4.03ch12,61100821,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AA,AG,AA,AA,AA,AA,AA,AA,AA,AA
SNP83827,G/A,ST4.03ch12,61100852,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,GA,GG,GG,GG,GG,GG,GG,GG,GG
SNP83828,T/A,ST4.03ch12,61100855,+,NaN,NaN,NaN,NaN,NaN,NaN,...,TT,TA,TT,TT,TT,TT,TT,TT,TT,TT


- Removiendo marcadores no informativos.

In [25]:
# Duplicados con segregación idéntica
def identical_segregation_filter(df):
    return df.drop_duplicates(subset=df.columns[10:])

In [26]:
preprocessed_df = identical_segregation_filter(preprocessed_df)
filter_summary.append(
    FilterDescription(
        "Marcadores únicos",
        len(preprocessed_df),
    )
)
preprocessed_df

,alleles,chrom,pos,strand,assembly,center,protLSID,assayLSID,panelLSID,QCcode,...,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
rs,,,,,,,,,,,,,,,,,,,,,
SNP1,C/T,ST4.03ch01,2597267,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP2,A/G,ST4.03ch01,2631717,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP3,C/A,ST4.03ch01,2631718,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
SNP4,G/A,ST4.03ch01,2631725,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,2631732,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNP83826,A/G,ST4.03ch12,61100821,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AA,AG,AA,AA,AA,AA,AA,AA,AA,AA
SNP83827,G/A,ST4.03ch12,61100852,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,GA,GG,GG,GG,GG,GG,GG,GG,GG
SNP83828,T/A,ST4.03ch12,61100855,+,NaN,NaN,NaN,NaN,NaN,NaN,...,TT,TA,TT,TT,TT,TT,TT,TT,TT,TT


- Removiendo genotipos raros.

No se cuenta con datos de calidad del genotipado, por lo que el MAF (Minor Allele Frequency) se calculara directamente de los genotipos llamados.

In [27]:
# Preserva los marcadores cuyo alelo menos frecuente esta en por lo menos 5% de la muestra
# Estimation of Minor allele frequency from called genotypes
def maf_from_called_filter(df: pd.DataFrame, threshold=0.05):
    def get_maf_from_called(row: pd.Series, threshold):
        allele_count = dict()
        total = 0
        for genotype, count in row[10:].value_counts().items():
            if genotype == "NN":
                continue
            total += 2 * count
            for allele in list(str(genotype)):
                if allele in allele_count:
                    allele_count[allele] += count
                else:
                    allele_count[allele] = count
        return min(allele_count.values()) / total >= threshold

    return df[df.apply(get_maf_from_called, axis=1, threshold=threshold)]

In [28]:
maf_threshold = 0.05
preprocessed_df = maf_from_called_filter(preprocessed_df, maf_threshold)
filter_summary.append(
    FilterDescription(
        f"Marcadores cuyo MAF >= {maf_threshold:.3f}",
        len(preprocessed_df),
    )
)
preprocessed_df

,alleles,chrom,pos,strand,assembly,center,protLSID,assayLSID,panelLSID,QCcode,...,G141,G142,G143,G144,G145,G146,G147,G148,G149,G150
rs,,,,,,,,,,,,,,,,,,,,,
SNP2,A/G,ST4.03ch01,2631717,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AG,AA,AA,AA,AA,AA,AA,AA,AA,AA
SNP4,G/A,ST4.03ch01,2631725,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GA,GG,GG,GG,GG,GG,GG,GG,GG,GG
SNP5,C/T,ST4.03ch01,2631732,+,NaN,NaN,NaN,NaN,NaN,NaN,...,CT,TT,TT,TT,TT,TT,TT,TT,CT,TT
SNP6,G/A,ST4.03ch01,2631734,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,GG,GG,GG,GG,GG,GG,GG,GA,GG
SNP7,A/G,ST4.03ch01,3683816,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SNP83825,A/G,ST4.03ch12,61100816,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AA,AG,AA,AA,AA,AA,AA,AA,AA,AA
SNP83826,A/G,ST4.03ch12,61100821,+,NaN,NaN,NaN,NaN,NaN,NaN,...,AA,AG,AA,AA,AA,AA,AA,AA,AA,AA
SNP83827,G/A,ST4.03ch12,61100852,+,NaN,NaN,NaN,NaN,NaN,NaN,...,GG,GA,GG,GG,GG,GG,GG,GG,GG,GG


In [29]:
print(*filter_summary, sep="\n")

Marcadores con frecuencia de datos faltantes (NN o sin datos) < 0.150: 83830
Marcadores únicos: 68189
Marcadores cuyo MAF >= 0.050: 45851
